 # Exercício 2 - Análise de Agregações com PySpark

 ## Objetivo
 Demonstrar técnicas de agregação e análise de dados usando PySpark

## Dataset

- Dados preparados do exercício anterior (formato Parquet)

## Técnicas aplicadas:

1. Agregações simples (count, avg, max, min)
2. Agregações com groupBy
3. Funções de janela (window functions)
4. Análise de distribuição temporal
5. Cálculo de métricas acumulativas

In [1]:
# Instalação do PySpark
!pip install pyspark

In [2]:
# Importação de bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
# Inicialização da sessão Spark
spark = SparkSession.builder \
    .appName("EBAC_Aggregation_Analysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

In [5]:
# 1. Leitura do arquivo Parquet (dados do exercício anterior)
df_video = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/spark/data/output/videos-preparados-parquet')

In [6]:
print("Schema dos dados carregados:")
df_video.printSchema()

print("\nPrimeiras 10 linhas:")
df_video.show(10, truncate=False)

Schema dos dados carregados:
root
 |-- Video ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Keyword_index: double (nullable = true)
 |-- Features: vector (nullable = true)
 |-- Features_Normal: vector (nullable = true)
 |-- Features_PCA: vector (nullable = true)


Primeiras 10 linhas:
+-----------+--------------------------------------------------------------------------------------------------+------------+-------+-----+--------+------+-----------+----+--------------------------------------------------------------------------

## Análise por Keyword

In [7]:
# 2. Quantidade de registros únicos para cada Keyword
print("Quantidade de registros únicos por Keyword:")
df_video_unicos = df_video.groupBy('keyword').agg(
    count_distinct('Video ID').alias('registros_unicos')
).orderBy(desc('registros_unicos'))

df_video_unicos.show()

Quantidade de registros únicos por Keyword:
+----------------+----------------+
|         keyword|registros_unicos|
+----------------+----------------+
|             cnn|              50|
|       interview|              50|
|          crypto|              50|
|    data science|              50|
|        trolling|              50|
|        tutorial|              50|
|          marvel|              50|
|game development|              50|
|         mrbeast|              50|
|         physics|              50|
|             sat|              49|
|         history|              49|
|           cubes|              49|
|        reaction|              49|
|          sports|              49|
|            asmr|              49|
|computer science|              48|
|            food|              48|
|          how-to|              48|
|machine learning|              48|
+----------------+----------------+
only showing top 20 rows



In [8]:
# 3. Média de Interaction por Keyword
print("Média de Interaction por Keyword:")
df_media_interaction = df_video.groupBy('keyword').agg(
    round(avg('Interaction'), 2).alias('media_interaction')
).orderBy(desc('media_interaction'))

df_media_interaction.show()

Média de Interaction por Keyword:
+-------------+-----------------+
|      keyword|media_interaction|
+-------------+-----------------+
|      animals|   1.0333333431E8|
|      mrbeast|    6.793032444E7|
|          bed|    5.108503355E7|
|        music|    2.756185246E7|
|        cubes|    1.602622671E7|
|      history|    1.576864797E7|
|        apple|    1.156792139E7|
|      mukbang|    1.129743534E7|
|       sports|       8864339.33|
|       how-to|       8659984.83|
|     business|       8034998.42|
|       marvel|        6712564.7|
|     tutorial|       6410616.21|
|         food|       5201958.73|
|      biology|       4301890.84|
|         lofi|       4088307.49|
|mathchemistry|        3838380.7|
|      physics|       3638337.31|
|       movies|       3486262.93|
|    interview|       2943255.47|
+-------------+-----------------+
only showing top 20 rows



In [9]:
# 4. Valor máximo de Interaction por Keyword (Ranking)
print("Ranking de Interaction máxima por Keyword:")
df_max_interaction = df_video.groupBy('keyword').agg(
    max('Interaction').alias('max_interaction')
).orderBy(desc('max_interaction'))

df_max_interaction.show()

Ranking de Interaction máxima por Keyword:
+--------+---------------+
| keyword|max_interaction|
+--------+---------------+
| animals|     1593623628|
|   music|      922551152|
|     bed|      532691631|
| history|      440187490|
|   apple|      429916936|
| mrbeast|      300397699|
|  google|      239385460|
|business|      210025196|
|   cubes|      170925917|
|  sports|      106924567|
| mukbang|       87433858|
|    lofi|       86445177|
|tutorial|       69616442|
|  movies|       65253870|
|  marvel|       56247330|
|  how-to|       53053975|
|    food|       48754479|
| physics|       43463298|
|    asmr|       34411125|
|nintendo|       32268486|
+--------+---------------+
only showing top 20 rows



## Análise Estatística de Views

In [15]:
# 5. Média e variância de Views por Keyword
print("Média e variância de Views por Keyword:")
df_media_var_views = df_video.groupBy('keyword').agg(
    round(avg('Views'), 2).alias('media_views'),
    round(variance('Views'), 2).alias('variancia_views')
).orderBy(desc('media_views'))

df_media_var_views.show()

Média e variância de Views por Keyword:
+-------------+--------------+--------------------+
|      keyword|   media_views|     variancia_views|
+-------------+--------------+--------------------+
|      animals|1.0247653421E8|9.112202469061308...|
|       google| 6.575379656E7|1.119365456341814...|
|      mrbeast| 6.571900681E7|3.960274041494829...|
|          bed| 5.065175547E7|9.857166664123382E15|
|        music| 2.719243194E7|1.609546048933449...|
|        cubes|   1.5722057E7|9.908543653889238E14|
|      history| 1.547669389E7|4.197693123865590...|
|        apple| 1.143489473E7|4.487154518237596E15|
|      mukbang| 1.114603738E7|5.667001759055671E14|
|       sports|    8770673.52|3.079555918324097E14|
|       how-to|    8485654.85|1.892768546871419...|
|     business|    7956048.78|1.009380832106395...|
|       marvel|    6493827.51|1.366887787045722...|
|     tutorial|    6252003.63|1.268746236063972...|
|         food|     5103661.5|7.015606103194809E13|
|      biology|    42307

In [22]:
# 6. Média, mínimo e máximo de Views por Keyword (formatado)
print("Estatísticas descritivas de Views por Keyword:")
df_media_min_max_views = df_video.groupBy('keyword').agg(
    format_number(avg('Views'), 0).alias('media_views'),
    format_number(min('Views'), 0).alias('min_views'),
    format_number(max('Views'), 0).alias('max_views')
).orderBy(desc('media_views',))

df_media_min_max_views.show()

Estatísticas descritivas de Views por Keyword:
+----------------+-----------+---------+-------------+
|         keyword|media_views|min_views|    max_views|
+----------------+-----------+---------+-------------+
|             sat|    898,606|    7,163|   18,116,954|
|      literature|    852,848|    2,847|    4,231,789|
|          sports|  8,770,674|      867|  106,014,469|
|          how-to|  8,485,655|    3,311|   52,061,447|
|         finance|    764,438|    1,195|    9,450,554|
|game development|    757,460|    1,352|    6,478,696|
|        business|  7,956,049|    2,270|  208,293,677|
|          google| 65,753,797|    8,064|2,147,483,647|
|         mrbeast| 65,719,007|  889,300|  285,526,909|
|          marvel|  6,493,828|    2,813|   54,583,132|
|        tutorial|  6,252,004|   19,323|   68,512,549|
|             cnn|    545,094|   51,269|    1,889,320|
|    data science|    516,893|      911|    3,069,097|
|             bed| 50,651,755|    4,454|  524,709,805|
|            food|

## Análise Temporal

In [23]:
# 7. Primeira e última data de publicação por Keyword
print("Primeira e última publicação por Keyword:")
df_first_last_publi = df_video.groupBy('Keyword').agg(
    first('Published At').alias('primeira_publicacao'),
    last('Published At').alias('ultima_publicacao')
).orderBy('Keyword')

df_first_last_publi.show()

Primeira e última publicação por Keyword:
+----------------+-------------------+-----------------+
|         Keyword|primeira_publicacao|ultima_publicacao|
+----------------+-------------------+-----------------+
|         animals|         2014-09-29|       2022-08-11|
|           apple|         2022-08-24|       2022-08-19|
|            asmr|         2022-08-24|       2022-08-22|
|             bed|         2017-04-17|       2018-06-21|
|         biology|         2020-07-16|       2012-06-04|
|        business|         2020-09-25|       2021-03-24|
|           chess|         2022-08-24|       2022-08-21|
|             cnn|         2022-08-24|       2022-08-05|
|computer science|         2017-02-28|       2021-08-18|
|          crypto|         2022-08-24|       2022-08-23|
|           cubes|         2022-08-23|       2021-07-26|
|    data science|         2018-12-04|       2020-09-17|
|       education|         2013-05-10|       2022-08-23|
|         finance|         2022-08-23|       2

## Análise de Duplicatas e Qualidade de Dados

In [24]:
# 8. Análise de duplicatas em Titles
print("Análise de duplicatas em Titles:")
df_contagem_titles = df_video.agg(
    count('title').alias('total_titles'),
    countDistinct('title').alias('titles_unicos')
)

Análise de duplicatas em Titles:


In [25]:
# Coletar resultados
resultado = df_contagem_titles.collect()[0]
total = resultado['total_titles']
unicos = resultado['titles_unicos']
diferenca = total - unicos

In [26]:
# Mostrar análise
print(f"📊 ANÁLISE DE TITLES:")
print(f"Total de titles: {total:,}")
print(f"Titles únicos: {unicos:,}")
print(f"Titles duplicados: {diferenca:,}")
print(f"Taxa de duplicação: {(diferenca/total)*100:.2f}%")

df_contagem_titles.show()

📊 ANÁLISE DE TITLES:
Total de titles: 15,070
Titles únicos: 1,851
Titles duplicados: 13,219
Taxa de duplicação: 87.72%
+------------+-------------+
|total_titles|titles_unicos|
+------------+-------------+
|       15070|         1851|
+------------+-------------+



## Análise Temporal Detalhada

In [27]:
# 9. Quantidade de registros por ano (ordem ascendente)
print("Quantidade de registros por ano:")
df_registros_por_ano = df_video.groupBy('Year') \
    .agg(count('*').alias('quantidade_registros')) \
    .orderBy('Year')

df_registros_por_ano.withColumn(
    'quantidade_registros',
    format_number('quantidade_registros', 0)
).show()

Quantidade de registros por ano:
+----+--------------------+
|Year|quantidade_registros|
+----+--------------------+
|2007|                  18|
|2008|                  10|
|2009|                  84|
|2010|                  54|
|2011|                  33|
|2012|                  91|
|2013|                  35|
|2014|                  70|
|2015|                 123|
|2016|                 274|
|2017|                 379|
|2018|                 440|
|2019|                 652|
|2020|               1,230|
|2021|               1,859|
|2022|               9,718|
+----+--------------------+



In [28]:
# 10. Quantidade de registros por ano e mês (ordem ascendente)
print("Quantidade de registros por ano e mês:")
df_registros_ano_mes = df_video.groupBy('Year', 'Month') \
    .agg(count('*').alias('quantidade_registros')) \
    .orderBy('Year', 'Month')

df_registros_ano_mes.withColumn(
    'quantidade_registros',
    format_number('quantidade_registros', 0)
).show(20)  # Mostrar mais linhas para melhor visualização

Quantidade de registros por ano e mês:
+----+-----+--------------------+
|Year|Month|quantidade_registros|
+----+-----+--------------------+
|2007|    7|                   9|
|2007|   12|                   9|
|2008|    7|                  10|
|2009|    2|                  19|
|2009|    6|                  17|
|2009|    7|                   9|
|2009|    8|                  10|
|2009|   10|                  10|
|2009|   12|                  19|
|2010|    3|                   9|
|2010|    5|                  20|
|2010|    6|                   9|
|2010|    9|                   9|
|2010|   10|                   7|
|2011|    2|                   9|
|2011|    5|                   9|
|2011|    9|                  10|
|2011|   10|                   5|
|2012|    1|                   9|
|2012|    2|                  24|
+----+-----+--------------------+
only showing top 20 rows



## Análise com Window Functions

In [29]:
# 11. Média acumulativa de Likes por Keyword ao longo dos anos
print("Calculando média acumulativa de Likes por Keyword...")

# Primeiro, agregar os likes por keyword e ano
df_likes_por_ano = df_video.groupBy('Keyword', 'Year') \
    .agg(avg('Likes').alias('media_likes_ano')) \
    .orderBy('Keyword', 'Year')

print("Média de Likes por Keyword e Ano:")
df_likes_por_ano.withColumn(
    'media_likes_ano',
    round('media_likes_ano', 2)
).show()

Calculando média acumulativa de Likes por Keyword...
Média de Likes por Keyword e Ano:
+-------+----+---------------+
|Keyword|Year|media_likes_ano|
+-------+----+---------------+
|animals|2009|      1357197.0|
|animals|2010|      196249.42|
|animals|2013|    1.1025176E7|
|animals|2014|     3520585.59|
|animals|2019|      1103713.0|
|animals|2020|      916438.49|
|animals|2021|      103950.95|
|animals|2022|       33579.43|
|  apple|2016|      4144389.0|
|  apple|2021|        38261.0|
|  apple|2022|       18625.55|
|   asmr|2020|       148120.0|
|   asmr|2021|      339891.27|
|   asmr|2022|       12444.36|
|    bed|2007|       317160.5|
|    bed|2009|      861904.43|
|    bed|2010|       144517.0|
|    bed|2011|      745097.21|
|    bed|2017|      372814.21|
|    bed|2018|      477308.03|
+-------+----+---------------+
only showing top 20 rows



In [30]:
# Definir a janela para cálculo acumulativo
window_spec = Window.partitionBy('Keyword') \
    .orderBy('Year') \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [31]:
# Calcular média acumulativa
df_media_acumulativa = df_likes_por_ano.withColumn(
    'media_acumulativa_likes',
    avg('media_likes_ano').over(window_spec)
)

In [32]:
# Formatar e mostrar resultados
print("Média acumulativa de Likes por Keyword:")
df_media_acumulativa \
    .withColumn('media_likes_ano', round('media_likes_ano', 2)) \
    .withColumn('media_acumulativa_likes', round('media_acumulativa_likes', 2)) \
    .orderBy('Keyword', 'Year') \
    .show(30)  # Mostrar mais resultados

Média acumulativa de Likes por Keyword:
+-------+----+---------------+-----------------------+
|Keyword|Year|media_likes_ano|media_acumulativa_likes|
+-------+----+---------------+-----------------------+
|animals|2009|      1357197.0|              1357197.0|
|animals|2010|      196249.42|              776723.21|
|animals|2013|    1.1025176E7|             4192874.14|
|animals|2014|     3520585.59|              4024802.0|
|animals|2019|      1103713.0|              3440584.2|
|animals|2020|      916438.49|             3019893.25|
|animals|2021|      103950.95|             2603330.06|
|animals|2022|       33579.43|             2282111.23|
|  apple|2016|      4144389.0|              4144389.0|
|  apple|2021|        38261.0|              2091325.0|
|  apple|2022|       18625.55|             1400425.18|
|   asmr|2020|       148120.0|               148120.0|
|   asmr|2021|      339891.27|              244005.64|
|   asmr|2022|       12444.36|              166818.55|
|    bed|2007|       3171

## 📈 Insights e Conclusões

In [33]:
# Resumo estatístico final
print("📈 RESUMO ESTATÍSTICO FINAL:")
print("=" * 50)

# Estatísticas gerais
total_videos = df_video.count()
keywords_unicos = df_video.select('keyword').distinct().count()

print(f"Total de vídeos analisados: {total_videos:,}")
print(f"Keywords únicos: {keywords_unicos}")
print(f"Média geral de Views: {df_video.agg(avg('Views')).collect()[0][0]:.0f}")
print(f"Média geral de Likes: {df_video.agg(avg('Likes')).collect()[0][0]:.0f}")
print(f"Média geral de Comments: {df_video.agg(avg('Comments')).collect()[0][0]:.0f}")

📈 RESUMO ESTATÍSTICO FINAL:
Total de vídeos analisados: 15,070
Keywords únicos: 41
Média geral de Views: 11293567
Média geral de Likes: 179633
Média geral de Comments: 8204


In [ ]:
# Encerramento da sessão Spark
spark.stop()
print("✅ Análise concluída e sessão Spark encerrada.")